# Load Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import joblib

import Models
from AuxiliarFunctions import *

2023-03-24 18:14:18.777818: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Load Data

In [2]:
config_file_path = 'data/config.csv'
df_config = pd.read_csv(config_file_path)
train_id = 0

df_config

,hash_id,label,raw_files_path,processed_file_folder,processed_file_path,cv_alg,cv_folds,cv_path,preproc_alg,pipeline_path,scaler_alg,train_data_path,train_trgt_path,model_path
0,6111500007297378247,Toy Data Classification with 10 StratifiedKFolds,data/raw,data,data/6111500007297378247_processed_data.csv,StratifiedKFolds,10,data/indexes,Não implementado para a aplicação!!!,data/pipelines,StandardScaler,data/6111500007297378247_train_data.csv,data/6111500007297378247_trgt_data.csv,data/models


In [3]:
df_data = pd.read_csv(df_config['train_data_path'][train_id])
df_trgt = pd.read_csv(df_config['train_trgt_path'][train_id])

# Processo de Treinamento de um modelo simples

In [33]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import copy

model_type = 'SVM'
n_folds =  df_config['cv_folds'][train_id]

cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]
model_path = df_config['model_path'][train_id]

print('Processing SVM Training')

kernels = ['linear']
regularizations = [0.5, 0.2]

train_record = None


for kernel in kernels:
    print('Processing Training for %s kernel'%(kernel))
    for regularization in regularizations:
        print('Processing Training for %s regularization'%(regularization))
        for ifold in range(n_folds):
            #if ifold >= 2: # para desenvolvimento
            #    break
            print('Training %i fold of %i folds\n\n'%(ifold+1, n_folds))

            print('Reading Cross-validation indexes')

            cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                               df_config['cv_alg'][train_id],
                                                               ifold, n_folds)
            print('Done')
            print('Reading Pipeline Object')
            with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
                [trn_idx,val_idx] = pickle.load(file_handler)

            pipe_name ='%s_%s_CV_fold_%i_of_%i_pipe.pkl'%(df_config['hash_id'][train_id],
                                                               df_config['cv_alg'][train_id],
                                                               ifold, n_folds)
            with open(os.path.join(pipe_path,pipe_name),'rb') as file_handler:
                pipe = joblib.load(file_handler)

            print('Done')

            trn_data = pipe.transform(df_data)
            trn_trgt = df_trgt.values # tf.keras.utils.to_categorical(dev_target, num_classes=len(np.unique(dev_target)))

            print('Training for model')
            model_name = '%s_%s_%i_fold_model_%s_kernel_%s_regularization.pkl'%(df_config['hash_id'][train_id],
                                                                                model_type, ifold, kernel, 
                                                                                str(regularization).replace('.','-'))

            if os.path.exists(os.path.join(model_path, model_name)):
                print('Model is in %s'%(os.path.join(model_path, model_name)))
            else:
                print('No Model \n\n')

                model = Models.SVMClassificationModel(kernel='linear', regularization=0.5, verbose=False)
                model.fit(trn_data, trn_trgt, trn_id=trn_idx, val_id=val_idx, random_state=0,)
                predictions = model.predict(trn_data)
                df_predict = pd.DataFrame(data=np.concatenate((trn_trgt, 
                                                               predictions[:,np.newaxis]),
                                                              axis=1), 
                                          columns=['target', 'model_output'])
                prediction_name = copy.copy(model_name)
                prediction_name = prediction_name.replace('.pkl','_prediction_file.csv')
                print(prediction_name)
                df_predict.to_csv(os.path.join(model_path, prediction_name),index=False)
               
                model.save(os.path.join(model_path, model_name))
                
                acc = Models.acc_score(df_predict.loc[trn_idx,'target'],
                                       df_predict.loc[trn_idx,'model_output'])
                sens = Models.sensitivity_score(df_predict.loc[trn_idx,'target'],
                                                df_predict.loc[trn_idx,'model_output'])
                spec = Models.specificity_score(df_predict.loc[trn_idx,'target'],
                                                df_predict.loc[trn_idx,'model_output'])
                sp = Models.sp_index(df_predict.loc[trn_idx,'target'],
                                     df_predict.loc[trn_idx,'model_output'])
                auc = Models.auc_score(df_predict.loc[trn_idx,'target'],
                                       df_predict.loc[trn_idx,'model_output'])
                
                dict_train_record = {
                    'hash_id':[df_config['hash_id'][train_id]],'fold':[ifold],
                    'prediction_file':[prediction_name], 'kernel':[kernel],
                    'regularization':[regularization], 'Acc':[acc],
                    'Sens':[sens],'Spec':[spec],'SP':[sp], 'AUC':[auc]
                }
                if train_record is None:
                    train_record = pd.DataFrame(data=dict_train_record)
                else:
                    train_record = pd.concat([train_record,pd.DataFrame(data=dict_train_record)],axis=0, ignore_index=True)
train_record.to_csv(os.path.join(model_path, 'train_record'),index=False)

Processing SVM Training
Processing Training for linear kernel
Processing Training for 0.5 regularization
Training 1 fold of 10 folds


Reading Cross-validation indexes
Done
Reading Pipeline Object
Done
Training for model
No Model 




/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


6111500007297378247_SVM_0_fold_model_linear_kernel_0-5_regularization_prediction_file.csv
Training 2 fold of 10 folds


Reading Cross-validation indexes
Done
Reading Pipeline Object
Done
Training for model
No Model 




/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


6111500007297378247_SVM_1_fold_model_linear_kernel_0-5_regularization_prediction_file.csv
Training 3 fold of 10 folds


Reading Cross-validation indexes
Done
Reading Pipeline Object
Done
Training for model
No Model 




...

6111500007297378247_SVM_7_fold_model_linear_kernel_0-2_regularization_prediction_file.csv
Training 9 fold of 10 folds


Reading Cross-validation indexes
Done
Reading Pipeline Object
Done
Training for model
No Model 




/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


6111500007297378247_SVM_8_fold_model_linear_kernel_0-2_regularization_prediction_file.csv
Training 10 fold of 10 folds


Reading Cross-validation indexes
Done
Reading Pipeline Object
Done
Training for model
No Model 




/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


6111500007297378247_SVM_9_fold_model_linear_kernel_0-2_regularization_prediction_file.csv


In [34]:
mean_grouped = train_record[['kernel','regularization',
                        'Acc','Sens','Spec','SP', 'AUC']].groupby(['kernel', 'regularization']).mean()
std_grouped = train_record[['kernel','regularization',
                        'Acc','Sens','Spec','SP', 'AUC']].groupby(['kernel', 'regularization']).std()
grouped = pd.concat([mean_grouped,std_grouped],axis=0)

In [35]:
grouped

Acc      Sens      Spec        SP       AUC
kernel regularization                                                  
linear 0.2             0.746500  0.780865  0.712094  0.556639  0.746479
       0.5             0.746500  0.780865  0.712094  0.556639  0.746479
       0.2             0.001304  0.002833  0.002777  0.001950  0.001306
       0.5             0.001304  0.002833  0.002777  0.001950  0.001306

In [32]:
train_record

,hash_id,fold,prediction_file,kernel,regularization,Acc,Sens,Spec,SP,AUC
0,6111500007297378247,0,6111500007297378247_SVM_0_fold_model_linear_ke...,linear,0.2,0.746889,0.783652,0.710093,0.557142,0.746873
1,6111500007297378247,1,6111500007297378247_SVM_1_fold_model_linear_ke...,linear,0.2,0.747778,0.781208,0.714317,0.558590,0.747763
2,6111500007297378247,2,6111500007297378247_SVM_2_fold_model_linear_ke...,linear,0.2,0.748000,0.779431,0.716541,0.558988,0.747986
3,6111500007297378247,3,6111500007297378247_SVM_3_fold_model_linear_ke...,linear,0.2,0.747000,0.779702,0.714254,0.557441,0.746978
4,6111500007297378247,4,6111500007297378247_SVM_4_fold_model_linear_ke...,linear,0.2,0.745222,0.775039,0.715366,0.554881,0.745202
5,6111500007297378247,5,6111500007297378247_SVM_5_fold_model_linear_ke...,linear,0.2,0.745667,0.780813,0.710474,0.555365,0.745643
6,6111500007297378247,6,6111500007297378247_SVM_6_fold_model_linear_ke...,linear,0.2,0.746778,0.781923,0.711586,0.557023,0.746754
7,6111500007297378247,7,6111500007297378247_SVM_7_fold_model_linear_ke...,linear,0.2,0.745000,0.780591,0.709362,0.554355,0.744976
8,6111500007297378247,8,6111500007297378247_SVM_8_fold_model_linear_ke...,linear,0.2,0.748111,0.785920,0.710251,0.558916,0.748086
9,6111500007297378247,9,6111500007297378247_SVM_9_fold_model_linear_ke...,linear,0.2,0.744556,0.780369,0.708695,0.553685,0.744532


# Treinamento de um modelo simples para decisão do médico

Durante o processo de desenvolvimento, tive uma discussão com o André sobre o processo de tomada de decisão de um médico e ele se baseia em 3 variáveis: proteina, citometria e dif cif. Ou seja, será que um modelo consegue fazer o mesmo que um médico apenas analisando estas variáveis?

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

model_type = 'DecisionTree'
n_folds =  df_config['cv_folds'][train_id]

cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]

n_folds = df_config['cv_folds'][train_id]

dataset_name = 'pos_test'

print(get_train_description(df_config, train_id))

print('Processing Decision Tree Training')
data = dev_pos_test
for ifold in range(n_folds):
    #if ifold != 0: # para desenvolvimento
    #    break

    print('Training %i fold of %i folds\n\n'%(ifold+1, n_folds))

    cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                     df_config['cv_alg'][train_id],
                                                     ifold, n_folds)
    with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
        [trn_idx,val_idx] = pickle.load(file_handler)

    pipe_name = '%s_%s_CV_fold_%i_of_%i_pipe_%s.jbl'%(df_config['hash_id'][train_id],
                                                      df_config['cv_alg'][train_id],
                                                      ifold, n_folds, 
                                                      dataset_name)
    with open(os.path.join(pipe_path,pipe_name),'rb') as file_handler:
        pipe = joblib.load(file_handler)

    trn_data = pipe.transform(data)
    trn_trgt = dev_target.values # tf.keras.utils.to_categorical(dev_target, num_classes=len(np.unique(dev_target)))
    print('Training for model')
    model_name = '%s_%s_%i_fold_model_%s.pkl'%(df_config['hash_id'][train_id],
                                               model_type, ifold, 
                                               dataset_name)
    model_path = df_config['model_path'][train_id]
    if os.path.exists(os.path.join(model_path, model_name)):
        print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
    else:
        print('Modelo não existe\n\n')
        model = DecisionTreeClassifier(max_leaf_nodes=4, random_state=0)
        model.fit(trn_data, trn_trgt)
        predictions = model.predict(trn_data)
        df_predict = pd.DataFrame(data=np.concatenate((dev_target.values, 
                                                       predictions[:,np.newaxis]),
                                                      axis=1), 
                                  columns=['target', 'model_output'])
        prediction_name = '%s_%s_%i_fold_prediction_file_%s.csv'%(df_config['hash_id'][train_id],
                                                                  model_type, ifold,
                                                                  dataset_name)
        df_predict.to_csv(os.path.join(model_path, prediction_name),index=False)
        
        pickle.dump(model, open(os.path.join(model_path, model_name), 'wb'))
        # to load: model = pickle.load(open((os.path.join(model_path, model_name), 'rb'))

## Análise Básica

In [ ]:
model_type = 'DecisionTree'

dataset_name = 'pos_test'#'personal', 'social', 'clinical', 'pos_test', 'PeSo', 'PeSoCli', 'PeCli', 'CliPos', 'PeSoCliPos_par'
figure_of_merit_names = ['sens', 'spec', 'acc', 'sp']

n_folds =  df_config['cv_folds'][train_id]
cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]
model_path = df_config['model_path'][train_id]

df_train = pd.DataFrame(data=np.zeros([df_config['cv_folds'][train_id],len(figure_of_merit_names)]), columns=figure_of_merit_names)
df_val = pd.DataFrame(np.zeros([df_config['cv_folds'][train_id],len(figure_of_merit_names)]))

for ifold in range(df_config['cv_folds'][train_id]):
    print('Processing fold %i of %i'%(ifold, df_config['cv_folds'][train_id]))
    
    cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                         df_config['cv_alg'][train_id],
                                                         ifold, n_folds)
    with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
        [trn_idx,val_idx] = pickle.load(file_handler)
    
    prediction_file_name = '%s_%s_%i_fold_prediction_file_%s.csv'%(df_config['hash_id'][train_id],
                                                                  model_type, ifold,
                                                                  dataset_name)
    print('Reading: ',prediction_file_name)
    df_predict = pd.read_csv(os.path.join(model_path, prediction_file_name))
    #print(df_predict['nn_output'].head())
    #print('\n')
    for figure_of_merit in figure_of_merit_names:
        
        if figure_of_merit == 'sens':
            df_train.loc[ifold,figure_of_merit] = Models.sensitivity_score(df_predict.loc[trn_idx,'target'], 
                                                                           df_predict.loc[trn_idx,'model_output'])
            df_val.loc[ifold,figure_of_merit] = Models.sensitivity_score(df_predict.loc[val_idx,'target'], 
                                                                         df_predict.loc[val_idx,'model_output'])
        if figure_of_merit == 'spec':
            df_train.loc[ifold,figure_of_merit] = Models.specificity_score(df_predict.loc[trn_idx,'target'], 
                                                                           df_predict.loc[trn_idx,'model_output'])
            df_val.loc[ifold,figure_of_merit] = Models.specificity_score(df_predict.loc[val_idx,'target'], 
                                                                         df_predict.loc[val_idx,'model_output'])
        if figure_of_merit == 'acc':
            df_train.loc[ifold,figure_of_merit] = Models.acc_score(df_predict.loc[trn_idx,'target'], 
                                                                   df_predict.loc[trn_idx,'model_output'])
            df_val.loc[ifold,figure_of_merit] = Models.acc_score(df_predict.loc[val_idx,'target'], 
                                                                         df_predict.loc[val_idx,'model_output'])
        if figure_of_merit == 'sp':
            df_train.loc[ifold,figure_of_merit] = Models.sp_index(df_predict.loc[trn_idx,'target'], 
                                                                  df_predict.loc[trn_idx,'model_output'])
            df_val.loc[ifold,figure_of_merit] = Models.sp_index(df_predict.loc[val_idx,'target'], 
                                                                df_predict.loc[val_idx,'model_output'])
df_fig_of_merit = pd.concat([df_train.mean(axis=0).T,df_train.std(axis=0).T],axis=1)
df_fig_of_merit = df_fig_of_merit.rename(columns={0:'mean',1:'std'}, )
df_fig_of_merit = df_fig_of_merit.T

In [ ]:
df_fig_of_merit

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

model_type = 'DecisionTree'

dataset_name = 'pos_test'#'personal', 'social', 'clinical', 'pos_test', 'PeSo', 'PeSoCli', 'PeCli', 'CliPos', 'PeSoCliPos_par'
figure_of_merit_names = ['sens', 'spec', 'acc', 'sp']

n_folds =  df_config['cv_folds'][train_id]
cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]
model_path = df_config['model_path'][train_id]

for ifold in range(df_config['cv_folds'][train_id]):
    print('Processing fold %i of %i'%(ifold, df_config['cv_folds'][train_id]))
    
    cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                         df_config['cv_alg'][train_id],
                                                         ifold, n_folds)
    with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
        [trn_idx,val_idx] = pickle.load(file_handler)
    
    model_name = '%s_%s_%i_fold_model_%s.pkl'%(df_config['hash_id'][train_id],
                                               model_type, ifold, 
                                               dataset_name)
    model = pickle.load(open(os.path.join(model_path, model_name), 'rb'))
    fn=list(dev_pos_test.columns)
    cn=['TB+', 'TB-']
    tree.plot_tree(model,feature_names = fn,class_names=cn,filled = True)
    plt.show()

In [ ]:
df_predict

In [ ]:
dev_target.shape

# Training Process

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

model_type = 'MLPNeuralNetwork'
n_folds =  df_config['cv_folds'][train_id]

cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]

n_folds = df_config['cv_folds'][train_id]
hidden_neurons = list(get_list_of_hidden_neurons(df_config['model_neurons'][train_id]))

datasets_data = [dev_social, dev_clinical, dev_pos_test, dev_personal]
datasets_name = ['social', 'clinical', 'pos_test', 'personal']

# para desenvolvimento
#datasets_data = [dev_social]
#datasets_name = ['social']


print(get_train_description(df_config, train_id))

for idataset, dataset_name in enumerate(datasets_name): 
    print('Processing %s'%(dataset_name))
    data = datasets_data[idataset]
    for ifold in range(n_folds):
        #if ifold != 0: # para desenvolvimento
        #    break
        
        print('Training %i fold of %i folds\n\n'%(ifold+1, n_folds))

        cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                         df_config['cv_alg'][train_id],
                                                         ifold, n_folds)
        with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
            [trn_idx,val_idx] = pickle.load(file_handler)

        pipe_name = '%s_%s_CV_fold_%i_of_%i_pipe_%s.jbl'%(df_config['hash_id'][train_id],
                                                          df_config['cv_alg'][train_id],
                                                          ifold, n_folds, 
                                                          datasets_name[idataset])
        with open(os.path.join(pipe_path,pipe_name),'rb') as file_handler:
            pipe = joblib.load(file_handler)

        trn_data = pipe.transform(data)
        trn_trgt = dev_target.values # tf.keras.utils.to_categorical(dev_target, num_classes=len(np.unique(dev_target)))
        for idx, ineuron in enumerate(hidden_neurons):
            print('Training for %i neuron in'%(ineuron),hidden_neurons)
            for iinit in range(df_config['model_inits'][train_id]):
                print('Training for %i init in %i inits'%(iinit+1, df_config['model_inits'][train_id]))
                model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                             model_type,ifold, 
                                                                             ineuron, iinit,
                                                                             datasets_name[idataset])
                model_path = df_config['model_path'][train_id]
                if os.path.exists(os.path.join(model_path, model_name)):
                    print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
                    #model = Models.MLPModel(n_hidden_neurons=ineuron,verbose=2)
                    #model.load(os.path.join(model_path, model_name))
                else:
                    print('Modelo não existe\n\n')
                    model = Models.MLPModel(n_hidden_neurons=ineuron,verbose=2)
                    model.fit(trn_data, trn_trgt, trn_id=trn_idx, val_id=val_idx, 
                              epochs=100,#df_config['model_epochs'][train_id], 
                              random_state=iinit, 
                              learning_rate=0.01,#df_config['model_learning_rate'][train_id],
                              patience=100,#df_config['model_patience'][train_id],
                              batch_size=int(df_config['model_batch_size'][train_id]),
                             )
                    predictions = model.predict(trn_data)
                    df_predict = pd.DataFrame(data=np.concatenate((dev_target.values, 
                                                                   predictions,
                                                                   np.sign(predictions)),
                                                                  axis=1), 
                                              columns=['target', 'nn_output', 'prediction'])
                    prediction_name = '%s_%s_%i_fold_%i_neuron_%i_init_prediction_file_%s.csv'%(df_config['hash_id'][train_id],
                                                                                                model_type,ifold, ineuron, iinit,
                                                                                                datasets_name[idataset])
                    df_predict.to_csv(os.path.join(model_path, prediction_name),index=False)
                    model.save(os.path.join(model_path, model_name))

# Análise de Topologia

In [ ]:
from sklearn.metrics import accuracy_score
from AuxiliarFunctions import *


print(get_train_description(df_config, train_id))


if True: # remova quando tiver segurança no treinamento
    model_type = 'MLPNeuralNetwork'
    
    datasets_data = [dev_social, dev_clinical, dev_pos_test, dev_personal]
    datasets_name = ['social', 'clinical', 'pos_test', 'personal']

    
    hidden_neurons = list(get_list_of_hidden_neurons(df_config['model_neurons'][train_id]))
    
    n_folds = df_config['cv_folds'][train_id]
    
    figure_of_merit_matrix = np.zeros([len(datasets_name),len(hidden_neurons),n_folds])
    best_init_matrix = np.zeros([len(datasets_name),len(hidden_neurons),n_folds])
    
    for idataset, dataset_name in enumerate(datasets_name): 
        print('Processing %s'%(dataset_name))
        data = datasets_data[idataset]
    
        for ifold in range(n_folds):
            print('Analysing %i fold of %i folds'%(ifold+1, n_folds))

            for idx, ineuron in enumerate(hidden_neurons):
                print('Analysing for %i neuron in'%(ineuron),hidden_neurons)

                best_figure_of_merit = -9999

                for iinit in range(df_config['model_inits'][train_id]):
                    print('Analysing for %i init in %i inits'%(iinit+1, df_config['model_inits'][train_id]))
                    # repare que mudou!
                    model_name = '%s_%s_%i_fold_%i_neuron_%i_init_prediction_file_%s.csv'%(df_config['hash_id'][train_id],
                                                                                           model_type,ifold, ineuron, iinit,
                                                                                           datasets_name[idataset])
                    model_path = df_config['model_path'][train_id]
                    #print(os.path.join(model_path, model_name))
                    if os.path.exists(os.path.join(model_path, model_name)):
                        print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
                        # adicionar a função de analise acc ou outra
                        df_predictions = pd.read_csv(os.path.join(model_path, model_name))
                        figure_of_merit = Models.acc_score(df_predictions['target'].values, 
                                                           df_predictions['prediction'].values)
                        print('Figure of Merit: Acc, value %1.3f'%(figure_of_merit))
                        if best_figure_of_merit < figure_of_merit:
                            figure_of_merit_matrix[idataset,idx, ifold] = figure_of_merit
                            best_figure_of_merit = figure_of_merit
                            best_init_matrix[idataset,idx,ifold] = iinit

                    else:
                        print('Modelo não existe\n\n')
                        # fora!
        

In [ ]:
import matplotlib.pyplot as plt

figure_of_merit_label = 'Accuracy'

for idataset, dataset_name in enumerate(datasets_name): 
    plt.figure(figsize=(6, 6))
    hidden_neurons = list(get_list_of_hidden_neurons(df_config['model_neurons'][train_id]))
    mean_figure_of_merit = np.mean(figure_of_merit_matrix[idataset,:,:],axis=1)
    std_figure_of_merit = np.std(figure_of_merit_matrix[idataset,:,:],axis=1) 

    linestyle = {"linestyle":"-", "linewidth":1, "markersize":2.5, "markeredgewidth":1, "elinewidth":1, "capsize":5}

    plt.errorbar(hidden_neurons, mean_figure_of_merit, yerr=std_figure_of_merit,
                 color="k", fmt='o',**linestyle, label='Figure of Merit')
    plt.title('%s - %s vs Neurons analysis'%(dataset_name,figure_of_merit_label))
    plt.ylabel('%s (percentage)'%(figure_of_merit_label))
    plt.xlabel('Number of Neurons')
    plt.grid()
    plt.show()

# Treinamento da Fusão - PeSo

In [ ]:
model_type = 'MLPNeuralNetwork'
n_folds =  df_config['cv_folds'][train_id]

cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]

hidden_neurons = list(get_list_of_hidden_neurons(df_config['model_neurons'][train_id]))

datasets_data = [dev_personal, dev_social]
datasets_name = ['personal', 'social']

# numero de neuronios de cada modelo que vai realizar a fusão
previous_neuron = [2,2]
previous_init = [0,0]
df_fusion = pd.DataFrame()

print(get_train_description(df_config, train_id))

for ifold in range(n_folds):
    #if ifold != 0: # para desenvolvimento
    #    break
    print('Training %i fold of %i folds\n\n'%(ifold+1, n_folds))
    cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                         df_config['cv_alg'][train_id],
                                                         ifold, n_folds)
    with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
        [trn_idx,val_idx] = pickle.load(file_handler)
        
    # agora eu tenho que fazer a saída de cada um dos modelos base e gerar um banco novo
    for idataset, dataset_name in enumerate(datasets_name):  
        print('Processing %s'%(dataset_name))
        data = datasets_data[idataset]
        # carregar o pipeline de processamento
        pipe_name = '%s_%s_CV_fold_%i_of_%i_pipe_%s.jbl'%(df_config['hash_id'][train_id],
                                                          df_config['cv_alg'][train_id],
                                                          ifold, n_folds, 
                                                          datasets_name[idataset])
        with open(os.path.join(pipe_path,pipe_name),'rb') as file_handler:
            pipe = joblib.load(file_handler)
        trn_data = pipe.transform(data)
        
        #carregar o modelo certo
        model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                     model_type,ifold, 
                                                                     previous_neuron[idataset], 
                                                                     previous_init[idataset],
                                                                     datasets_name[idataset])
        model_path = df_config['model_path'][train_id]
        if os.path.exists(os.path.join(model_path, model_name)):
            print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
            model = Models.MLPModel(n_hidden_neurons=previous_neuron[idataset],verbose=2)
            model.load(os.path.join(model_path, model_name))
            processed_data = model.predict_one_layer_before_output(trn_data)
            df_buffer = pd.DataFrame(data=processed_data)
            df_buffer = df_buffer.add_prefix('%s_'%(dataset_name))
        
        if idataset == 0:
            df_fusion = df_buffer
        else:
            df_fusion = pd.concat([df_fusion, df_buffer],axis=1)
            
    trn_data = df_fusion.values
    trn_trgt = dev_target.values
    dataset_name = 'PeSo'
    
    for idx, ineuron in enumerate(hidden_neurons):
        print('Training for %i neuron in'%(ineuron),hidden_neurons)
        for iinit in range(df_config['model_inits'][train_id]):
            print('Training for %i init in %i inits'%(iinit+1, df_config['model_inits'][train_id]))
            model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                         model_type,ifold, 
                                                                         ineuron, iinit,
                                                                         dataset_name)
            if os.path.exists(os.path.join(model_path, model_name)):
                print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
            else:
                print('Modelo não existe\n\n')
                model = Models.MLPModel(n_hidden_neurons=ineuron,verbose=2)
                model.fit(trn_data, trn_trgt, trn_id=trn_idx, val_id=val_idx, 
                          epochs=1000,#df_config['model_epochs'][train_id], 
                          random_state=iinit, 
                          learning_rate=0.01,#df_config['model_learning_rate'][train_id],
                          patience=10,#df_config['model_patience'][train_id],
                          batch_size=int(df_config['model_batch_size'][train_id]),
                         )
                predictions = model.predict(trn_data)
                df_predict = pd.DataFrame(data=np.concatenate((dev_target.values, 
                                                               predictions,
                                                               np.sign(predictions)),
                                                              axis=1), 
                                          columns=['target', 'nn_output', 'prediction'])
                prediction_name = '%s_%s_%i_fold_%i_neuron_%i_init_prediction_file_%s.csv'%(df_config['hash_id'][train_id],
                                                                                            model_type,ifold, ineuron, iinit,
                                                                                            dataset_name)
                df_predict.to_csv(os.path.join(model_path, prediction_name),index=False)
                model.save(os.path.join(model_path, model_name))

# Treinamento de Fusão - PeSoCli

In [ ]:
model_type = 'MLPNeuralNetwork'
n_folds =  df_config['cv_folds'][train_id]

cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]

hidden_neurons = list(get_list_of_hidden_neurons(df_config['model_neurons'][train_id]))

datasets_data = [dev_personal, dev_social]
datasets_name = ['personal', 'social']

# numero de neuronios de cada modelo que vai realizar a fusão
PeSo_neurons = [2,2]
PeSo_inits = [0,0]
df_PeSo_fusion = pd.DataFrame()
df_PeSo_data = [dev_personal, dev_social]
df_PeSo_names = ['personal', 'social']


PeSoCli_neurons = [2,3]
PeSoCli_inits = [0,0]
df_PeSoCli_fusion = pd.DataFrame()

model_path = df_config['model_path'][train_id]

print(get_train_description(df_config, train_id))

for ifold in range(n_folds):
    #if ifold != 0: # para desenvolvimento
    #    break
        
    df_buffer = []
    print('Training %i fold of %i folds\n\n'%(ifold+1, n_folds))
    cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                         df_config['cv_alg'][train_id],
                                                         ifold, n_folds)
    with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
        [trn_idx,val_idx] = pickle.load(file_handler)
        
    # agora eu tenho que fazer a saída de cada um dos modelos base e gerar um banco novo
    for idataset, dataset_name in enumerate(df_PeSo_names):  
        print('Processing %s'%(dataset_name))
        data = datasets_data[idataset]
        # carregar o pipeline de processamento
        pipe_name = '%s_%s_CV_fold_%i_of_%i_pipe_%s.jbl'%(df_config['hash_id'][train_id],
                                                          df_config['cv_alg'][train_id],
                                                          ifold, n_folds, 
                                                          datasets_name[idataset])
        with open(os.path.join(pipe_path,pipe_name),'rb') as file_handler:
            pipe = joblib.load(file_handler)
        trn_data = pipe.transform(data)
        
        #carregar o modelo certo
        model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                     model_type,ifold, 
                                                                     previous_neuron[idataset], 
                                                                     previous_init[idataset],
                                                                     datasets_name[idataset])
        
        if os.path.exists(os.path.join(model_path, model_name)):
            print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
            model = Models.MLPModel(n_hidden_neurons=previous_neuron[idataset],verbose=2)
            model.load(os.path.join(model_path, model_name))
            processed_data = model.predict_one_layer_before_output(trn_data)
            df_buffer = pd.DataFrame(data=processed_data)
            df_buffer = df_buffer.add_prefix('%s_'%(dataset_name))
        
        if idataset == 0:
            df_PeSo_fusion = df_buffer
        else:
            df_PeSo_fusion = pd.concat([df_PeSo_fusion, df_buffer],axis=1)
 
    dataset_name = 'PeSo'   
    model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                 model_type,ifold, 
                                                                 PeSoCli_neurons[0],
                                                                 PeSoCli_inits[0],
                                                                 dataset_name)
    
    df_buffer = []
    if os.path.exists(os.path.join(model_path, model_name)):
        print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
        model = Models.MLPModel(n_hidden_neurons=PeSoCli_neurons[0],verbose=2)
        model.load(os.path.join(model_path, model_name))
        processed_data = model.predict_one_layer_before_output(df_PeSo_fusion.values)
        df_buffer = pd.DataFrame(data=processed_data)
        df_buffer = df_buffer.add_prefix('%s_'%(dataset_name))
        df_PeSoCli_fusion = df_buffer
    df_buffer = []
    
    dataset_name = 'clinical'   
    model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                 model_type,ifold, 
                                                                 PeSoCli_neurons[1],
                                                                 PeSoCli_inits[1],
                                                                 dataset_name)
    
    
    if os.path.exists(os.path.join(model_path, model_name)):
        print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
        
        data = dev_clinical
        # carregar o pipeline de processamento
        pipe_name = '%s_%s_CV_fold_%i_of_%i_pipe_%s.jbl'%(df_config['hash_id'][train_id],
                                                          df_config['cv_alg'][train_id],
                                                          ifold, n_folds, 
                                                          dataset_name)
        with open(os.path.join(pipe_path,pipe_name),'rb') as file_handler:
            pipe = joblib.load(file_handler)
        trn_data = pipe.transform(data)
        
        model = Models.MLPModel(n_hidden_neurons=PeSoCli_neurons[1],verbose=2)
        model.load(os.path.join(model_path, model_name))
        processed_data = model.predict_one_layer_before_output(trn_data)
        df_buffer = pd.DataFrame(data=processed_data)
        df_buffer = df_buffer.add_prefix('%s_'%(dataset_name))
        df_PeSoCli_fusion = pd.concat([df_PeSoCli_fusion, df_buffer],axis=1)
        
        
    trn_data = df_PeSoCli_fusion.values
    trn_trgt = dev_target.values
    dataset_name = 'PeSoCli'
    
    for idx, ineuron in enumerate(hidden_neurons):
        print('Training for %i neuron in'%(ineuron),hidden_neurons)
        for iinit in range(df_config['model_inits'][train_id]):
            print('Training for %i init in %i inits'%(iinit+1, df_config['model_inits'][train_id]))
            model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                         model_type,ifold, 
                                                                         ineuron, iinit,
                                                                         dataset_name)
            if os.path.exists(os.path.join(model_path, model_name)):
                print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
            else:
                print('Modelo não existe\n\n')
                model = Models.MLPModel(n_hidden_neurons=ineuron,verbose=2)
                model.fit(trn_data, trn_trgt, trn_id=trn_idx, val_id=val_idx, 
                          epochs=1000,#df_config['model_epochs'][train_id], 
                          random_state=iinit, 
                          learning_rate=0.01,#df_config['model_learning_rate'][train_id],
                          patience=10,#df_config['model_patience'][train_id],
                          batch_size=int(df_config['model_batch_size'][train_id]),
                         )
                predictions = model.predict(trn_data)
                df_predict = pd.DataFrame(data=np.concatenate((dev_target.values, 
                                                               predictions,
                                                               np.sign(predictions)),
                                                              axis=1), 
                                          columns=['target', 'nn_output', 'prediction'])
                prediction_name = '%s_%s_%i_fold_%i_neuron_%i_init_prediction_file_%s.csv'%(df_config['hash_id'][train_id],
                                                                                            model_type,ifold, ineuron, iinit,
                                                                                            dataset_name)
                df_predict.to_csv(os.path.join(model_path, prediction_name),index=False)
                model.save(os.path.join(model_path, model_name))

# Treinamento de Fusão - PeCli

In [ ]:
model_type = 'MLPNeuralNetwork'
n_folds =  df_config['cv_folds'][train_id]

cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]

hidden_neurons = list(get_list_of_hidden_neurons(df_config['model_neurons'][train_id]))

datasets_data = [dev_personal, dev_clinical]
datasets_name = ['personal', 'clinical']

# numero de neuronios de cada modelo que vai realizar a fusão
previous_neuron = [2,3]
previous_init = [0,0]
df_fusion = pd.DataFrame()

print(get_train_description(df_config, train_id))

for ifold in range(n_folds):
    #if ifold != 0: # para desenvolvimento
    #    break
    print('Training %i fold of %i folds\n\n'%(ifold+1, n_folds))
    cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                         df_config['cv_alg'][train_id],
                                                         ifold, n_folds)
    with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
        [trn_idx,val_idx] = pickle.load(file_handler)
        
    # agora eu tenho que fazer a saída de cada um dos modelos base e gerar um banco novo
    for idataset, dataset_name in enumerate(datasets_name):  
        print('Processing %s'%(dataset_name))
        data = datasets_data[idataset]
        # carregar o pipeline de processamento
        pipe_name = '%s_%s_CV_fold_%i_of_%i_pipe_%s.jbl'%(df_config['hash_id'][train_id],
                                                          df_config['cv_alg'][train_id],
                                                          ifold, n_folds, 
                                                          datasets_name[idataset])
        with open(os.path.join(pipe_path,pipe_name),'rb') as file_handler:
            pipe = joblib.load(file_handler)
        trn_data = pipe.transform(data)
        
        #carregar o modelo certo
        model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                     model_type,ifold, 
                                                                     previous_neuron[idataset], 
                                                                     previous_init[idataset],
                                                                     datasets_name[idataset])
        model_path = df_config['model_path'][train_id]
        if os.path.exists(os.path.join(model_path, model_name)):
            print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
            model = Models.MLPModel(n_hidden_neurons=previous_neuron[idataset],verbose=2)
            model.load(os.path.join(model_path, model_name))
            processed_data = model.predict_one_layer_before_output(trn_data)
            df_buffer = pd.DataFrame(data=processed_data)
            df_buffer = df_buffer.add_prefix('%s_'%(dataset_name))
        
        if idataset == 0:
            df_fusion = df_buffer
        else:
            df_fusion = pd.concat([df_fusion, df_buffer],axis=1)
            
    trn_data = df_fusion.values
    trn_trgt = dev_target.values
    dataset_name = 'PeCli'
    
    for idx, ineuron in enumerate(hidden_neurons):
        print('Training for %i neuron in'%(ineuron),hidden_neurons)
        for iinit in range(df_config['model_inits'][train_id]):
            print('Training for %i init in %i inits'%(iinit+1, df_config['model_inits'][train_id]))
            model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                         model_type,ifold, 
                                                                         ineuron, iinit,
                                                                         dataset_name)
            if os.path.exists(os.path.join(model_path, model_name)):
                print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
            else:
                print('Modelo não existe\n\n')
                model = Models.MLPModel(n_hidden_neurons=ineuron,verbose=2)
                model.fit(trn_data, trn_trgt, trn_id=trn_idx, val_id=val_idx, 
                          epochs=1000,#df_config['model_epochs'][train_id], 
                          random_state=iinit, 
                          learning_rate=0.01,#df_config['model_learning_rate'][train_id],
                          patience=10,#df_config['model_patience'][train_id],
                          batch_size=int(df_config['model_batch_size'][train_id]),
                         )
                predictions = model.predict(trn_data)
                df_predict = pd.DataFrame(data=np.concatenate((dev_target.values, 
                                                               predictions,
                                                               np.sign(predictions)),
                                                              axis=1), 
                                          columns=['target', 'nn_output', 'prediction'])
                prediction_name = '%s_%s_%i_fold_%i_neuron_%i_init_prediction_file_%s.csv'%(df_config['hash_id'][train_id],
                                                                                            model_type,ifold, ineuron, iinit,
                                                                                            dataset_name)
                df_predict.to_csv(os.path.join(model_path, prediction_name),index=False)
                model.save(os.path.join(model_path, model_name))

# Treinamento de Fusão - CliPos

In [ ]:
model_type = 'MLPNeuralNetwork'
n_folds =  df_config['cv_folds'][train_id]

cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]

hidden_neurons = list(get_list_of_hidden_neurons(df_config['model_neurons'][train_id]))

datasets_data = [dev_clinical, dev_pos_test]
datasets_name = ['clinical', 'pos_test']

# numero de neuronios de cada modelo que vai realizar a fusão
previous_neuron = [3,2]
previous_init = [0,0]
df_fusion = pd.DataFrame()

print(get_train_description(df_config, train_id))

for ifold in range(n_folds):
    #if ifold != 0: # para desenvolvimento
    #    break
    print('Training %i fold of %i folds\n\n'%(ifold+1, n_folds))
    cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                         df_config['cv_alg'][train_id],
                                                         ifold, n_folds)
    with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
        [trn_idx,val_idx] = pickle.load(file_handler)
        
    # agora eu tenho que fazer a saída de cada um dos modelos base e gerar um banco novo
    for idataset, dataset_name in enumerate(datasets_name):  
        print('Processing %s'%(dataset_name))
        data = datasets_data[idataset]
        # carregar o pipeline de processamento
        pipe_name = '%s_%s_CV_fold_%i_of_%i_pipe_%s.jbl'%(df_config['hash_id'][train_id],
                                                          df_config['cv_alg'][train_id],
                                                          ifold, n_folds, 
                                                          datasets_name[idataset])
        with open(os.path.join(pipe_path,pipe_name),'rb') as file_handler:
            pipe = joblib.load(file_handler)
        trn_data = pipe.transform(data)
        
        #carregar o modelo certo
        model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                     model_type,ifold, 
                                                                     previous_neuron[idataset], 
                                                                     previous_init[idataset],
                                                                     datasets_name[idataset])
        model_path = df_config['model_path'][train_id]
        if os.path.exists(os.path.join(model_path, model_name)):
            print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
            model = Models.MLPModel(n_hidden_neurons=previous_neuron[idataset],verbose=2)
            model.load(os.path.join(model_path, model_name))
            processed_data = model.predict_one_layer_before_output(trn_data)
            df_buffer = pd.DataFrame(data=processed_data)
            df_buffer = df_buffer.add_prefix('%s_'%(dataset_name))
        
        if idataset == 0:
            df_fusion = df_buffer
        else:
            df_fusion = pd.concat([df_fusion, df_buffer],axis=1)
            
    trn_data = df_fusion.values
    trn_trgt = dev_target.values
    dataset_name = 'CliPos'
    
    for idx, ineuron in enumerate(hidden_neurons):
        print('Training for %i neuron in'%(ineuron),hidden_neurons)
        for iinit in range(df_config['model_inits'][train_id]):
            print('Training for %i init in %i inits'%(iinit+1, df_config['model_inits'][train_id]))
            model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                         model_type,ifold, 
                                                                         ineuron, iinit,
                                                                         dataset_name)
            if os.path.exists(os.path.join(model_path, model_name)):
                print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
            else:
                print('Modelo não existe\n\n')
                model = Models.MLPModel(n_hidden_neurons=ineuron,verbose=2)
                model.fit(trn_data, trn_trgt, trn_id=trn_idx, val_id=val_idx, 
                          epochs=1000,#df_config['model_epochs'][train_id], 
                          random_state=iinit, 
                          learning_rate=0.01,#df_config['model_learning_rate'][train_id],
                          patience=10,#df_config['model_patience'][train_id],
                          batch_size=int(df_config['model_batch_size'][train_id]),
                         )
                predictions = model.predict(trn_data)
                df_predict = pd.DataFrame(data=np.concatenate((dev_target.values, 
                                                               predictions,
                                                               np.sign(predictions)),
                                                              axis=1), 
                                          columns=['target', 'nn_output', 'prediction'])
                prediction_name = '%s_%s_%i_fold_%i_neuron_%i_init_prediction_file_%s.csv'%(df_config['hash_id'][train_id],
                                                                                            model_type,ifold, ineuron, iinit,
                                                                                            dataset_name)
                df_predict.to_csv(os.path.join(model_path, prediction_name),index=False)
                model.save(os.path.join(model_path, model_name))

# Treinamento de Fusão - PeSoCliPos (Cascateado)

# Treinamento de Fusão - PeSoCliPos (Paralelo)

In [ ]:
model_type = 'MLPNeuralNetwork'
n_folds =  df_config['cv_folds'][train_id]

cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]

hidden_neurons = list(get_list_of_hidden_neurons(df_config['model_neurons'][train_id]))

datasets_data = [dev_personal, dev_social, dev_clinical, dev_pos_test]
datasets_name = ['personal', 'social', 'clinical', 'pos_test']

# numero de neuronios de cada modelo que vai realizar a fusão
previous_neuron = [2, 2, 3, 2]
previous_init = [0, 0, 0, 0]
df_fusion = pd.DataFrame()

print(get_train_description(df_config, train_id))

for ifold in range(n_folds):
    #if ifold != 0: # para desenvolvimento
    #    break
    print('Training %i fold of %i folds\n\n'%(ifold+1, n_folds))
    cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                         df_config['cv_alg'][train_id],
                                                         ifold, n_folds)
    with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
        [trn_idx,val_idx] = pickle.load(file_handler)
        
    # agora eu tenho que fazer a saída de cada um dos modelos base e gerar um banco novo
    for idataset, dataset_name in enumerate(datasets_name):  
        print('Processing %s'%(dataset_name))
        data = datasets_data[idataset]
        # carregar o pipeline de processamento
        pipe_name = '%s_%s_CV_fold_%i_of_%i_pipe_%s.jbl'%(df_config['hash_id'][train_id],
                                                          df_config['cv_alg'][train_id],
                                                          ifold, n_folds, 
                                                          datasets_name[idataset])
        with open(os.path.join(pipe_path,pipe_name),'rb') as file_handler:
            pipe = joblib.load(file_handler)
        trn_data = pipe.transform(data)
        
        #carregar o modelo certo
        model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                     model_type,ifold, 
                                                                     previous_neuron[idataset], 
                                                                     previous_init[idataset],
                                                                     datasets_name[idataset])
        model_path = df_config['model_path'][train_id]
        if os.path.exists(os.path.join(model_path, model_name)):
            print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
            model = Models.MLPModel(n_hidden_neurons=previous_neuron[idataset],verbose=2)
            model.load(os.path.join(model_path, model_name))
            processed_data = model.predict_one_layer_before_output(trn_data)
            df_buffer = pd.DataFrame(data=processed_data)
            df_buffer = df_buffer.add_prefix('%s_'%(dataset_name))
        
        if idataset == 0:
            df_fusion = df_buffer
        else:
            df_fusion = pd.concat([df_fusion, df_buffer],axis=1)
            
    trn_data = df_fusion.values
    trn_trgt = dev_target.values
    dataset_name = 'PeSoCliPos_par'
    
    for idx, ineuron in enumerate(hidden_neurons):
        print('Training for %i neuron in'%(ineuron),hidden_neurons)
        for iinit in range(df_config['model_inits'][train_id]):
            print('Training for %i init in %i inits'%(iinit+1, df_config['model_inits'][train_id]))
            model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model_%s.pkl'%(df_config['hash_id'][train_id],
                                                                         model_type,ifold, 
                                                                         ineuron, iinit,
                                                                         dataset_name)
            if os.path.exists(os.path.join(model_path, model_name)):
                print('Modelo existente em %s'%(os.path.join(model_path, model_name)))
            else:
                print('Modelo não existe\n\n')
                model = Models.MLPModel(n_hidden_neurons=ineuron,verbose=2)
                model.fit(trn_data, trn_trgt, trn_id=trn_idx, val_id=val_idx, 
                          epochs=1000,#df_config['model_epochs'][train_id], 
                          random_state=iinit, 
                          learning_rate=0.01,#df_config['model_learning_rate'][train_id],
                          patience=10,#df_config['model_patience'][train_id],
                          batch_size=int(df_config['model_batch_size'][train_id]),
                         )
                predictions = model.predict(trn_data)
                df_predict = pd.DataFrame(data=np.concatenate((dev_target.values, 
                                                               predictions,
                                                               np.sign(predictions)),
                                                              axis=1), 
                                          columns=['target', 'nn_output', 'prediction'])
                prediction_name = '%s_%s_%i_fold_%i_neuron_%i_init_prediction_file_%s.csv'%(df_config['hash_id'][train_id],
                                                                                            model_type,ifold, ineuron, iinit,
                                                                                            dataset_name)
                df_predict.to_csv(os.path.join(model_path, prediction_name),index=False)
                model.save(os.path.join(model_path, model_name))

# Análise Básica

In [ ]:
model_type = 'MLPNeuralNetwork'

ineuron = 5
iinit = 0
dataset_name = 'PeCli'#'personal', 'social', 'clinical', 'pos_test', 'PeSo', 'PeSoCli', 'PeCli', 'CliPos', 'PeSoCliPos_par'
figure_of_merit_names = ['sens', 'spec', 'acc', 'sp']

n_folds =  df_config['cv_folds'][train_id]
cv_path = df_config['cv_path'][train_id]
pipe_path = df_config['pipeline_path'][train_id]
model_path = df_config['model_path'][train_id]

df_train = pd.DataFrame(data=np.zeros([df_config['cv_folds'][train_id],len(figure_of_merit_names)]), columns=figure_of_merit_names)
df_val = pd.DataFrame(np.zeros([df_config['cv_folds'][train_id],len(figure_of_merit_names)]))

for ifold in range(df_config['cv_folds'][train_id]):
    print('Processing fold %i of %i'%(ifold, df_config['cv_folds'][train_id]))
    
    cv_name = '%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                         df_config['cv_alg'][train_id],
                                                         ifold, n_folds)
    with open(os.path.join(cv_path,cv_name),'rb') as file_handler:
        [trn_idx,val_idx] = pickle.load(file_handler)
    
    prediction_file_name = '%s_%s_%i_fold_%i_neuron_%i_init_prediction_file_%s.csv'%(df_config['hash_id'][train_id],
                                                                                     model_type,ifold, ineuron, iinit,
                                                                                     dataset_name)
    print('Reading: ',prediction_file_name)
    df_predict = pd.read_csv(os.path.join(model_path, prediction_file_name))
    #print(df_predict['nn_output'].head())
    #print('\n')
    for figure_of_merit in figure_of_merit_names:
        
        if figure_of_merit == 'sens':
            df_train.loc[ifold,figure_of_merit] = Models.sensitivity_score(df_predict.loc[trn_idx,'target'], 
                                                                           df_predict.loc[trn_idx,'prediction'])
            df_val.loc[ifold,figure_of_merit] = Models.sensitivity_score(df_predict.loc[val_idx,'target'], 
                                                                         df_predict.loc[val_idx,'prediction'])
        if figure_of_merit == 'spec':
            df_train.loc[ifold,figure_of_merit] = Models.specificity_score(df_predict.loc[trn_idx,'target'], 
                                                                           df_predict.loc[trn_idx,'prediction'])
            df_val.loc[ifold,figure_of_merit] = Models.specificity_score(df_predict.loc[val_idx,'target'], 
                                                                         df_predict.loc[val_idx,'prediction'])
        if figure_of_merit == 'acc':
            df_train.loc[ifold,figure_of_merit] = Models.acc_score(df_predict.loc[trn_idx,'target'], 
                                                                   df_predict.loc[trn_idx,'prediction'])
            df_val.loc[ifold,figure_of_merit] = Models.acc_score(df_predict.loc[val_idx,'target'], 
                                                                         df_predict.loc[val_idx,'prediction'])
        if figure_of_merit == 'sp':
            df_train.loc[ifold,figure_of_merit] = Models.sp_index(df_predict.loc[trn_idx,'target'], 
                                                                  df_predict.loc[trn_idx,'prediction'])
            df_val.loc[ifold,figure_of_merit] = Models.sp_index(df_predict.loc[val_idx,'target'], 
                                                                df_predict.loc[val_idx,'prediction'])
df_fig_of_merit = pd.concat([df_train.mean(axis=0).T,df_train.std(axis=0).T],axis=1)
df_fig_of_merit = df_fig_of_merit.rename(columns={0:'mean',1:'std'}, )
df_fig_of_merit = df_fig_of_merit.T

In [ ]:
df_fig_of_merit